In [1]:
from pyzfn import Pyzfn as op           # 'op' function to open the .zarr files
from pyzfn import utils                 # other various utilities
import numpy as np                      # working with vector data
from matplotlib import pyplot as plt    # plotting graphs
import cmocean                          # pretty colormaps
utils.load_mpl_style()                  # pretty graphs
import ipywidgets

First, we will use the `helpers.py` file to keep our functions so they can be easily accessible. Copy the `plot_2d_magnetization` function to that file. Then, run the following cell to import it.

In [2]:
# This following block import helper.py
# In there, put functions that you will reuse
# If you create a new function, you need to rerun this cell
# if you modify a function, you don't need to rerun it.
import sys
sys.path.insert(0, '../')
import helpers

The function can now be called as part of the `helpers` module like the following:

In [ ]:
helpers.plot_2d_magnetization()

### Task 2
Now, we will use a function that will plot a cross section and the 2D data, given;
 - `path`: The path to the simulation data i.e. `./simulations/task1.zarr`
 - `dset`: The dataset i.e. `m_a`, `m_b` ...
 - `c`: The component of the magnetization which is `0` for $m_x$, `1` for $m_y$ and `2` for $m_z$ 
 - `yslice`: The index of the y value of the cross-section that you want between 0 and `Ny`

Then, run the function for `m_a`, `m_b`, `m_c` and `m_d`

/!\ Careful: No variables that were defined outside the function must be used inside the function or you will regret it later !

In [ ]:
def cross_section(path:str,dset:str,c:int, y_slice:int):
    fig, (ax1,ax2) = plt.subplots(2,1,figsize=(10,6),sharex=True) 
    task = op(path) # open the .zarr file
    cross_section = task[dset][0,0,y_slice,:,c] # we select t=0, z=0, y=50 and c=1 but we keep all data for x
    x = np.arange(cross_section.shape[0]) * task.dx * 1e9

    ax1.plot(x,cross_section) # use the `plot` function to display 1D data.
    ax1.set_ylim(-1,1) # set the y axis limits
    ax1.set_ylabel(r"$m_y$") # adding a label for the y axis of the plot, we are plotting m_y in nm
    ax2.set_xlabel(r"$x$ (nm)") # adding a label for the x axis of the plot, we are plotting m_y in nm
    ax2.set_ylabel(r"$y$ (nm)") # adding a label for the y axis of the plot, we are plotting m_y in nm
    ax1.grid(True) # add a grid to the plot
    ax2.grid(True) # add a grid to the plot
    
    xyplane = task[dset][0,0,:,:,c] # we select t=0, z=0, and c=0 but we keep all data for y and x (the xyplane)
    im = ax2.imshow( # plot the magnetization component
        task[dset][0,0,:,:,c],  # `mag` is a numpy array
        cmap=cmocean.cm.balance,  # pretty colormap
        origin="lower", # put the origin at the bottom left
        extent=[0, task.Tx*1e9, 0, task.Ty*1e9], # set the x and y axis limits ( in nm )
        vmin=-1, # set the color scale limits ( reducded magnetization is from -1 to 1 )
        vmax=1 # set the color scale limits ( reducded magnetization is from -1 to 1 )
    )

    ax2.axhline(y_slice * task.dx * 1e9,ls='--',c='k') # we had a dashed line on the image to see where x=20 is
    cax = ax2.inset_axes([1.01,0,0.02,1],transform=ax2.transAxes)
    cax.grid(False)
    fig.colorbar(im,cax=cax)
    cax.set_ylabel(r"$m_y$")


Changing all of these values can be tedious, we can use "widgets" to create an interactive plot.
Now, `dset`, `c` and `y_slice` are given through sliders and drop downs

In [ ]:
def interactive_cross_section(path:str):
    fig, (ax1,ax2) = plt.subplots(2,1,figsize=(10,6),sharex=True) 
    task = op(path) # open the .zarr file

    def plot(y_slice:int, dset:str, c:int):
        ax1.clear()
        ax2.clear()
        ax2.axhline(y_slice * task.dx * 1e9,ls='--',c='k') # we had a dashed line on the image to see where x=20 is
        cross_section = task[dset][0,0,y_slice,:,c] # we select t=0, z=0, y=50 and c=1 but we keep all data for x
        x = np.arange(cross_section.shape[0]) * task.dx * 1e9

        ax1.plot(x,cross_section) # use the `plot` function to display 1D data.
        ax1.set_ylim(-1,1) # set the y axis limits
        ax1.set_ylabel(rf"$m_{['x','y','z'][c]}$") # adding a label for the y axis of the plot, we are plotting m_y in nm
        ax2.set_xlabel(r"$x$ (nm)") # adding a label for the x axis of the plot, we are plotting m_y in nm
        ax2.set_ylabel(r"$y$ (nm)") # adding a label for the y axis of the plot, we are plotting m_y in nm
        ax1.grid(True) # add a grid to the plot
        ax2.grid(True) # add a grid to the plot
        
        xyplane = task[dset][0,0,:,:,c] # we select t=0, z=0, and c=0 but we keep all data for y and x (the xyplane)
        im = ax2.imshow( # plot the magnetization component
            task[dset][0,0,:,:,c],  # `mag` is a numpy array
            cmap=cmocean.cm.balance,  # pretty colormap
            origin="lower", # put the origin at the bottom left
            extent=[0, task.Tx*1e9, 0, task.Ty*1e9], # set the x and y axis limits ( in nm )
            vmin=-1, # set the color scale limits ( reducded magnetization is from -1 to 1 )
            vmax=1 # set the color scale limits ( reducded magnetization is from -1 to 1 )
        )
        cax = ax2.inset_axes([1.01,0,0.02,1],transform=ax2.transAxes)
        cax.grid(False)
        fig.colorbar(im,cax=cax)
        cax.set_ylabel(r"$m_y$")

    ipywidgets.interact(
        plot,
        dset=list(task.keys()), 
        c=[('mx', 0), ('my', 1), ('mz', 2)], 
        y_slice=ipywidgets.IntSlider(min=0, max=100, step=1, value=50)
    )

interactive_cross_section("./simulations/task1.zarr")

Add these 2 functions to `helpers.py` 

### Periodic boundary conditions

Calculate static magnetic configuration in a 1000 nm long and 200 nm wide and 5 nm thick permalloy stripe assuming periodic boundary conditions with 10 repetitions along the $x$ axis (it will allow you to study magnetic configuration in an infinitely long stripe).
* To implement periodic boundary conditions (PBC) with 10 repetitions along the $x$-axis, set the appropriate variable: `PBCx=10`. This must be placed before `setgeom()` !
* Use exactly the same parameters as in `task1`.
* Perform calculations for the same field values as `task1`:

 - Make the modifications asked above in `task2.mx3` and run it.
 - Using the `helpers.interactive_cross_section` function, observe the differences in magnetization between `task1` and `task2`

In [ ]:
# code here ...

 - Set both `PBCx=10` and `PBCy=10`. 
 - Observe the differences.

In [1]:
# code here ...